In [1]:
import os
import cv2
import numpy as np

In [2]:
def list_files_in_directory(directory):
    """List all files in a given directory."""
    files = os.listdir(directory)
    files = [directory + file for file in files if(os.path.isfile(os.path.join(directory, file)))]
    return sorted(files)
            

In [35]:
sim_world_avatar = list_files_in_directory('sim_frames_enlight/')
sim_world_noavatar = list_files_in_directory('sim_frames_enlight_world4/')
world_mask = cv2.imread('enlightenment_world_mask.png').astype(np.float32)/255.0
world_in_head = cv2.imread('enlightenment_orig_noself.png').astype(np.float32)/255.0

In [4]:
# cv2.imshow('Combined Image', cv2.imread(sim_world_avatar[100]))
cv2.imshow('Combined Image', world_mask)
cv2.waitKey(0)

102

In [5]:
img_sim_world_avatar = cv2.imread(sim_world_avatar[-1]).astype(np.float32)/255.0
img_sim_world_noavatar = cv2.imread(sim_world_noavatar[-1]).astype(np.float32)/255.0

print(img_sim_world_avatar.shape)
print(img_sim_world_noavatar.shape)
print(world_mask.shape)

(1180, 1180, 3)
(1180, 1180, 3)
(1388, 1388, 3)


In [19]:
# Center the mask in the image
# The width of the image
ew = 91
newshape = (img_sim_world_avatar.shape[0] - 2*ew, img_sim_world_avatar.shape[1] - 2*ew)


world_mask_rs = np.zeros_like(img_sim_world_avatar)
world_mask_rs[ew:-ew, ew:-ew, :] = cv2.resize(world_mask, newshape)

# Same with the original world
world_in_head_rs = np.ones_like(img_sim_world_avatar)
world_in_head_rs[ew:-ew, ew:-ew, :] = cv2.resize(world_in_head, newshape)

In [ ]:
cv2.imshow('Combined Image', img_sim_world_noavatar)
cv2.waitKey(0)

In [ ]:
# Fade between world sims and the world mask
alpha = 0.5
comb = alpha * img_sim_world_avatar + (1-alpha) * img_sim_world_noavatar
cv2.imshow('Combined Image', comb)
cv2.waitKey(0)

In [ ]:
beta = 0.8
masked_comb = (beta*world_mask_rs)*comb + world_mask_rs*(1-beta)*world_in_head_rs + (1 - world_mask_rs)*world_in_head_rs
cv2.imshow('Combined Image', masked_comb)
cv2.waitKey(0)

In [26]:

beta = 0.8
ranger = 1000

# Iterate through all files and combine them
for i in range(len(sim_world_avatar)):
    alpha = 1 - float(i)/ranger
    img_sim_world_avatar = cv2.imread(sim_world_avatar[i]).astype(np.float32)/255.0
    img_sim_world_noavatar = cv2.imread(sim_world_noavatar[i]).astype(np.float32)/255.0
    
    comb = alpha * img_sim_world_avatar + (1-alpha) * img_sim_world_noavatar
    masked_comb = (beta*world_mask_rs)*comb + world_mask_rs*(1-beta)*world_in_head_rs + (1 - world_mask_rs)*world_in_head_rs
    cv2.imshow('Combined Image', masked_comb)
    cv2.waitKey(10)
    print('Showing image : ', i)
    if(i > ranger):
        break

Showing image :  0
Showing image :  1
Showing image :  2
Showing image :  3
Showing image :  4
Showing image :  5
Showing image :  6
Showing image :  7
Showing image :  8
Showing image :  9
Showing image :  10
Showing image :  11
Showing image :  12
Showing image :  13
Showing image :  14
Showing image :  15
Showing image :  16
Showing image :  17
Showing image :  18
Showing image :  19
Showing image :  20
Showing image :  21
Showing image :  22
Showing image :  23
Showing image :  24
Showing image :  25
Showing image :  26
Showing image :  27
Showing image :  28
Showing image :  29
Showing image :  30
Showing image :  31
Showing image :  32
Showing image :  33
Showing image :  34
Showing image :  35
Showing image :  36
Showing image :  37
Showing image :  38
Showing image :  39
Showing image :  40
Showing image :  41
Showing image :  42
Showing image :  43
Showing image :  44
Showing image :  45
Showing image :  46
Showing image :  47
Showing image :  48
Showing image :  49
Showing im

Showing image :  398
Showing image :  399
Showing image :  400
Showing image :  401
Showing image :  402
Showing image :  403
Showing image :  404
Showing image :  405
Showing image :  406
Showing image :  407
Showing image :  408
Showing image :  409
Showing image :  410
Showing image :  411
Showing image :  412
Showing image :  413
Showing image :  414
Showing image :  415
Showing image :  416
Showing image :  417
Showing image :  418
Showing image :  419
Showing image :  420
Showing image :  421
Showing image :  422
Showing image :  423
Showing image :  424
Showing image :  425
Showing image :  426
Showing image :  427
Showing image :  428
Showing image :  429
Showing image :  430
Showing image :  431
Showing image :  432
Showing image :  433
Showing image :  434
Showing image :  435
Showing image :  436
Showing image :  437
Showing image :  438
Showing image :  439
Showing image :  440
Showing image :  441
Showing image :  442
Showing image :  443
Showing image :  444
Showing image

IndexError: list index out of range

In [36]:
runsum = None
count = 0
for img in sim_world_noavatar:
    if(count == 0):
        runsum = cv2.imread(img).astype(np.float32)/255.0
    else:
        runsum += cv2.imread(img).astype(np.float32)/255.0
    count += 1
    #print(count, img, runsum, cv2.imread(img))

runsum /= count

cv2.imshow('Combined Image', runsum)
cv2.waitKey(10)


-1